In [85]:
import boto3
import json

## Lambda

In [86]:
lambda_client = boto3.client('lambda')

In [87]:
def get_applicable_lambdas():
    tags_per_func = {}
    for response in lambda_client.get_paginator('list_functions').paginate():
        for function in response['Functions']:
            name = function['FunctionName']
            tags = lambda_client.get_function(FunctionName=name).get('Tags', {})
            tags_per_func[name] = tags
    return tags_per_func

In [88]:
lambda_functions = get_applicable_lambdas()

# SQS

In [89]:
sqs_client = boto3.client('sqs')

In [90]:
def get_applicable_queues():
    tags_per_queue = {}
    for response in sqs_client.get_paginator('list_queues').paginate():
        for queue in response['QueueUrls']:
            tags = sqs_client.list_queue_tags(QueueUrl=queue).get('Tags', {})
            tags_per_queue[queue] = tags
    return tags_per_queue

In [210]:
sqs_queues = get_applicable_queues()

# DynamoDB

In [92]:
dynamo_client = boto3.client('dynamodb')

In [93]:
tags_per_table = {}
for response in dynamo_client.get_paginator('list_tables').paginate():
    for table in response['TableNames']:
        arn = dynamo_client.describe_table(TableName=table)['Table']['TableArn']
        tags = dynamo_client.list_tags_of_resource(ResourceArn=arn)['Tags']
        tags_per_table[table] = tags
        
print(tags_per_table)

{'devChecklists': [{'Key': 'STAGE', 'Value': 'dev'}]}


# Kinesis

In [212]:
kinesis_client = boto3.client('kinesis')

In [213]:
tags_per_stream = {}
for response in kinesis_client.get_paginator('list_streams').paginate():
    for stream_name in response['StreamNames']:
        stream_tags = kinesis_client.list_tags_for_stream(StreamName=stream_name)['Tags']
        tags_per_stream[stream_name] = stream_tags

In [214]:
tags_per_stream

{}

# Create dashboard

In [97]:
cw_client = boto3.client('cloudwatch')

In [118]:
from aws_cdk.aws_cloudwatch import Dashboard, GraphWidget

In [182]:
METRIC_PERIOD = 60
REGION = boto3.DEFAULT_SESSION.region_name
WIDGET_WIDTH = 24
WIDGET_HEIGHT = 6
MAX_WIDTH = 24
DASHBOARD_PERIOD = '-PT3H'

In [183]:
def create_metric_widget(title, metrics):
    return dict(
        type='metric',
        properties=dict(
            metrics=metrics,
            view='timeSeries',
            region=REGION,
            stat='Maximum',
            period=METRIC_PERIOD,
            title=title
        )        
    )

In [195]:
def create_all_lambda_widget():
    all_lambda_metrics = [
        ['AWS/Lambda', 'Duration', {'stat': 'Maximum'}],
        ['.', '.', {'stat': 'Average'}],
        ['.', '.', {'stat': 'p95'}],
        ['.', '.', {'stat': 'Minimum'}],
        ['.', 'ConcurrentExecutions', {'stat': 'Maximum', 'yAxis': 'right'}]
    ]
    return create_metric_widget(title='All Functions Duration/Concurrent Executions', metrics=all_lambda_metrics)

In [196]:
from collections import namedtuple

In [197]:
Metric = namedtuple('Metric', ['metric', 'stat'])

In [198]:
func_metrics = [
    Metric('Duration', 'Average'),
    Metric('Duration', 'p95'),
    Metric('Duration', 'Maximum'),
    Metric('Invocations', 'Sum'),
    Metric('ConcurrentExecutions', 'Maximum')
]

In [199]:
def create_func_metrics(functions, metric):
    return [
        ['AWS/Lambda', metric.metric, 'FunctionName', func, {'stat': metric.stat }]
        for func in functions
    ]

In [211]:
func_widgets = []
def create_function_widgets():
    return [
        create_metric_widget(
            title=f'{metric.metric} {metric.stat} per Function', 
            metrics=create_func_metrics(lambda_functions, metric) 
        ) for metric in func_metrics
    ]

In [201]:
widgets = [create_all_lambda_widget()] + create_function_widgets()

In [202]:
x = 0
y = 0

In [208]:
for widget in widgets:
    if x + WIDGET_WIDTH > MAX_WIDTH:
        y += WIDGET_HEIGHT
        x = 0
    widget['x'] = x
    widget['y'] = y
    widget['width'] = WIDGET_WIDTH
    widget['height'] = WIDGET_HEIGHT
    x += WIDGET_WIDTH

In [209]:
dash = {
    'start': DASHBOARD_PERIOD,
    'widgets': widgets
}
dash_body = json.dumps(dash)

In [207]:
dash_response = cw_client.put_dashboard(DashboardName='SLICWatch', DashboardBody=dash_body)
print(dash_response)

{'DashboardValidationMessages': [], 'ResponseMetadata': {'RequestId': 'a61f6c5a-e8df-4d94-ae5e-4ecd6a40c69e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a61f6c5a-e8df-4d94-ae5e-4ecd6a40c69e', 'content-type': 'text/xml', 'content-length': '292', 'date': 'Mon, 20 Jul 2020 10:50:19 GMT'}, 'RetryAttempts': 0}}
